### A company engaged in the wholesale supply of coffee beans requested a monthly sales report. The interest is only in those sales that did not have delivery: their number per month, the percentage of bad reviews, the profitability of such transactions - in terms of sales with and without the client entering a promotional code. There is a download of all sales (Транзакции.csv) since the founding of the company. 
#### The task is to write code in Python that will carry out the necessary calculations and save the result table into Excel. 
##### Summary table fields: «Месяц» – «Введен промокод» – «Кол-во транзакций» – «Плохих отзывов, %» - «Доходность, %».

In [63]:
import pandas as pd

In [64]:
transactions = pd.read_csv('Транзакции.csv', sep='\t', encoding='cp1251')
transactions.head()

,TransactionID,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,0,0,"2605,0",5481,0,0
1,2,2019-Sep-3 11:53:03,90,8307,1,1,"8553,0",5956,1,1
2,3,2019-Oct-15 4:31:02,59,9819,0,1,"9995,0",5170,1,0
3,4,2019-Aug-26 4:14:00,14,5351,1,1,"12603,0",8826,0,0
4,5,2020-Aug-27 1:21:03,14,7779,1,0,"5625,0",5357,0,0


In [65]:
transactions.isnull().sum()

TransactionID    0
Date             0
Count            0
Cost             0
Доставка         0
Плохой отзыв     0
Paid             0
Weight           0
Promo            0
AutoIssue        0
dtype: int64

In [66]:
transactions.dtypes

TransactionID     int64
Date             object
Count             int64
Cost              int64
Доставка          int64
Плохой отзыв      int64
Paid             object
Weight            int64
Promo             int64
AutoIssue         int64
dtype: object

In [67]:
#Type float for 'Paid'
float_func = lambda x: float(x.replace(',', '.'))
transactions.Paid = transactions.Paid.map(float_func)

In [68]:
transactions = transactions[transactions.Доставка == 0]
transactions.head()

,TransactionID,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug-21 8:53:03,67,6325,0,0,2605.0,5481,0,0
2,3,2019-Oct-15 4:31:02,59,9819,0,1,9995.0,5170,1,0
7,8,2020-Aug-23 13:34:03,45,6381,0,0,6251.0,9227,0,0
8,9,2020-Aug-31 16:57:02,31,8795,0,1,2047.0,6044,0,1
9,10,2020-Aug-29 9:18:01,41,4175,0,1,6300.0,7866,1,1


In [69]:
first8_func = lambda x: x[:8]
transactions.Date = transactions['Date'].map(first8_func)

In [70]:
transactions.head()

,TransactionID,Date,Count,Cost,Доставка,Плохой отзыв,Paid,Weight,Promo,AutoIssue
0,1,2020-Aug,67,6325,0,0,2605.0,5481,0,0
2,3,2019-Oct,59,9819,0,1,9995.0,5170,1,0
7,8,2020-Aug,45,6381,0,0,6251.0,9227,0,0
8,9,2020-Aug,31,8795,0,1,2047.0,6044,0,1
9,10,2020-Aug,41,4175,0,1,6300.0,7866,1,1


In [71]:
grouped = transactions.groupby(['Date', 'Promo'])

In [72]:
final_df = grouped[['Count', 'Плохой отзыв', 'Cost', 'Paid']].sum()

In [73]:
final_df.head()

Count  Плохой отзыв     Cost        Paid
Date     Promo                                          
2019-Apr 0       8932            88  1234569  1330813.00
         1       7115            64   942324  1003110.00
2019-Aug 0       9756            82  1268800  1339810.34
         1       7508            84  1032339  1127964.00
2019-Dec 0      11602           113  1603016  1735341.00

In [74]:
final_df['Всего_отзывов'] = grouped['Плохой отзыв'].count()

In [75]:
final_df['Плохих отзывов, %'] = final_df['Плохой отзыв']/final_df['Всего_отзывов']*100

In [76]:
final_df = final_df.drop(['Плохой отзыв', 'Всего_отзывов'], axis = 1)

In [77]:
final_df['Доходность, %'] = (final_df.Cost - final_df.Paid)/final_df.Cost*100

In [78]:
final_df = final_df.drop(['Cost', 'Paid'], axis = 1)

In [79]:
final_df = final_df.reset_index()

In [80]:
final_df = final_df.rename(columns = {'Date':'Месяц', 'Promo':'Введен промокод',
                                 'Count':'Кол-во транзакций'})

In [81]:
round_func = lambda x: round(x, 2)
final_df['Плохих отзывов, %'] = final_df['Плохих отзывов, %'].map(round_func)
final_df['Доходность, %'] = final_df['Доходность, %'].map(round_func)

In [82]:
final_df.head()

,Месяц,Введен промокод,Кол-во транзакций,"Плохих отзывов, %","Доходность, %"
0,2019-Apr,0,8932,49.72,-7.80
1,2019-Apr,1,7115,46.04,-6.45
2,2019-Aug,0,9756,44.81,-5.60
3,2019-Aug,1,7508,57.14,-9.26
4,2019-Dec,0,11602,50.22,-8.25


In [88]:
final_df.to_excel('1. Кофе/out.xlsx', index = False, encoding='cp1251')